## Implementation of PilotNet, with variation

Implement [PilotNet](https://arxiv.org/pdf/1604.07316.pdf) using Keras (with theano backend), while also trying different variations

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import keras

from keras.models import Sequential, Model
from keras.layers import Input, SpatialDropout1D, Concatenate
from keras.layers import Flatten, Dense, Dropout, Lambda
from keras.layers import BatchNormalization
from keras.layers import deserialize
from keras.layers import dot, add, concatenate
from keras.layers.convolutional import *

from keras.optimizers import SGD, RMSprop, Adam, Nadam
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.preprocessing import image, sequence
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint

from keras.models import load_model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [3]:
from resnet import resnet

### Check data format

check the csv output, and shuffle

In [4]:
%ls ../data/csv/final

v1.csv  v2.csv  v3.csv  v3_train.csv  v3_valid.csv


In [5]:
os.path.dirname(os.getcwd())

'/home/im9uri/PycharmProjects/europilot'

In [6]:
parent_path = os.path.dirname(os.getcwd())

data_path = os.path.join(parent_path, 'data')
img_front_dir_path = os.path.join(data_path, 'img', 'front')
model_path = os.path.join(parent_path, 'model')
log_path = os.path.join(model_path, 'log')


csv_dir_path = os.path.join(data_path, 'csv', 'final')
cur_file = 'v3'
train_file = os.path.join(csv_dir_path, cur_file + '_train.csv')
valid_file = os.path.join(csv_dir_path, cur_file + '_valid.csv')

In [7]:
df_train = pd.read_csv(os.path.join(data_path, train_file))
print("%d rows" % df_train.shape[0])
df_train.head(3)

129996 rows


,img,wheel-axis,clutch,brake,gas,paddle-left,paddle-right,wheel-button-left-1,wheel-button-left-2,wheel-button-left-3,...,gear-1,gear-2,gear-3,gear-4,gear-5,gear-6,gear-R,front,side_left,side_right
0,9d0c3c2b_2017_07_27_14_56_31_97.jpg,-321,0,0,65535,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9d0c3c2b_2017_07_27_14_56_31_97_front.jpg,9d0c3c2b_2017_07_27_14_56_31_97_left.jpg,9d0c3c2b_2017_07_27_14_56_31_97_right.jpg
1,7d590ce8_2017_08_07_14_49_16_12.jpg,-741,27091,0,65535,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7d590ce8_2017_08_07_14_49_16_12_front.jpg,7d590ce8_2017_08_07_14_49_16_12_left.jpg,7d590ce8_2017_08_07_14_49_16_12_right.jpg
2,3f80cca8_2017_08_08_14_39_00_36.jpg,152,15222,0,65535,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3f80cca8_2017_08_08_14_39_00_36_front.jpg,3f80cca8_2017_08_08_14_39_00_36_left.jpg,3f80cca8_2017_08_08_14_39_00_36_right.jpg


In [8]:
df_val = pd.read_csv(os.path.join(data_path, valid_file))
print("%d rows" % df_val.shape[0])
df_val.head(3)

32499 rows


,img,wheel-axis,clutch,brake,gas,paddle-left,paddle-right,wheel-button-left-1,wheel-button-left-2,wheel-button-left-3,...,gear-1,gear-2,gear-3,gear-4,gear-5,gear-6,gear-R,front,side_left,side_right
0,7a13935b_2017_07_30_15_38_28_68.jpg,-41,0,0,45153,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7a13935b_2017_07_30_15_38_28_68_front.jpg,7a13935b_2017_07_30_15_38_28_68_left.jpg,7a13935b_2017_07_30_15_38_28_68_right.jpg
1,9d0c3c2b_2017_07_28_18_09_10_29.jpg,2000,0,0,65535,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9d0c3c2b_2017_07_28_18_09_10_29_front.jpg,9d0c3c2b_2017_07_28_18_09_10_29_left.jpg,9d0c3c2b_2017_07_28_18_09_10_29_right.jpg
2,15be80cb_2017_07_28_22_48_24_52.jpg,-1593,0,0,63730,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15be80cb_2017_07_28_22_48_24_52_front.jpg,15be80cb_2017_07_28_22_48_24_52_left.jpg,15be80cb_2017_07_28_22_48_24_52_right.jpg


###  Predict steering angle, using front image

define a generator that loops through the data

In [9]:
def img_to_arr(p):
    with image.load_img(p) as img:
        img = image.img_to_array(img)
    return img

def generator(img_list, wheel_axis, batch_size, img_shape):
    
    batch_img = np.zeros((batch_size,) + img_shape)
    batch_label = np.zeros((batch_size, 1))
    
    index = 0
    while True:
        for i in range(batch_size):
            img_name = img_list[index]
            arr = img_to_arr(os.path.join(img_front_dir_path, img_name))
            arr /= 255 # TODO: subtract by mean
            batch_img[i] = arr
            batch_label[i] = wheel_axis[index]
            
            index += 1
            if index == len(img_list):
                # loop over data
                index = 0
            
        yield batch_img, batch_label
    

In [10]:
input_shape = img_to_arr(os.path.join(img_front_dir_path, df_train['front'][0])).shape
batch_size = 64
train_steps = (df_train.shape[0] / batch_size) + 1
val_steps = (df_val.shape[0] / batch_size) + 1

print("input_shape: %s, batch_size: %d, train_steps: %d, val_steps: %d" % 
      (input_shape, batch_size, train_steps, val_steps))

input_shape: (3, 341, 562), batch_size: 64, train_steps: 2032, val_steps: 508


In [11]:
train_batch = generator(df_train['front'], df_train['wheel-axis'], batch_size, input_shape)
val_batch = generator(df_val['front'], df_val['wheel-axis'], batch_size, input_shape)

### Define model

In [12]:
# test resnet
model = resnet(input_shape=input_shape, filter_size=32)
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam', loss='mse')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 341, 562)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 3, 347, 568)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 64, 171, 281)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 171, 281)  256         conv1[0][0]                      
___________________________________________________________________________________________

bn3b_branch2b (BatchNormalizatio (None, 64, 43, 70)    256         res3b_branch2b[0][0]             
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 64, 43, 70)    0           bn3b_branch2b[0][0]              
____________________________________________________________________________________________________
add_4 (Add)                      (None, 64, 43, 70)    0           activation_12[0][0]              
                                                                   activation_10[0][0]              
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 64, 43, 70)    0           add_4[0][0]                      
____________________________________________________________________________________________________
res4a_branch2a (Conv2D)          (None, 128, 22, 35)   73856       activation_13[0][0]     

Total params: 2,822,881
Trainable params: 2,817,953
Non-trainable params: 4,928
____________________________________________________________________________________________________


In [12]:
# test nvidia model
def get_model(input_shape):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=input_shape),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    
    return model

model = get_model(input_shape)
adam = Adam(lr=1e-4, decay=1e-6)
model.compile(optimizer=adam, loss="mse") 
model.summary()

In [12]:
# or load from saved model
model = load_model('../model/v3-resnet50-000-208450.h5')

### Train

목표: val loss가 100,000 보다 내려가기

In [13]:
# define callbacks
cur_model = cur_file + '-resnet_v1'
csv_logger = CSVLogger(os.path.join(log_path, cur_model + '.log'))
reduce_lr = ReduceLROnPlateau(patience=5, verbose=1, min_lr=1e-07)

model_file_name= os.path.join(model_path, cur_model + '-{epoch:03d}-{val_loss:.0f}.h5')
checkpoint = ModelCheckpoint(model_file_name, verbose=0, save_best_only=True)

In [ ]:
model.fit_generator(train_batch, train_steps, epochs=50, verbose=1, callbacks=[csv_logger, checkpoint],
                    validation_data=val_batch, validation_steps=val_steps, initial_epoch=0)

Epoch 1/50
 362/2032 [====>.........................] - ETA: 1304s - loss: 920436.3957

In [15]:
model.fit_generator(train_batch, train_steps, epochs=50, verbose=1, callbacks=[csv_logger, checkpoint],
                    validation_data=val_batch, validation_steps=val_steps, initial_epoch=4)

Epoch 5/50
1300/1300 [==============================] - 821s - loss: 252657.3481 - val_loss: 344327.9345
Epoch 6/50
1300/1300 [==============================] - 821s - loss: 242017.7801 - val_loss: 342743.3952
Epoch 7/50
1300/1300 [==============================] - 820s - loss: 231311.2176 - val_loss: 345065.5769
Epoch 8/50
1300/1300 [==============================] - 821s - loss: 231372.9541 - val_loss: 342914.9102
Epoch 9/50
1300/1300 [==============================] - 825s - loss: 228675.4309 - val_loss: 349544.9034
Epoch 10/50
1300/1300 [==============================] - 826s - loss: 220107.8464 - val_loss: 350107.9660
Epoch 11/50
1300/1300 [==============================] - 827s - loss: 220093.6346 - val_loss: 341751.3201
Epoch 12/50
1300/1300 [==============================] - 825s - loss: 219377.9941 - val_loss: 345116.5716
Epoch 13/50
1300/1300 [==============================] - 825s - loss: 214599.9988 - val_loss: 339230.8396
Epoch 14/50
1300/1300 [============================

In [16]:
model.save('../model/v3-50-343369.h5')

In [18]:
model.fit_generator(train_batch, train_steps, epochs=20, verbose=1, callbacks=[csv_logger, checkpoint],
                    validation_data=val_batch, validation_steps=val_steps, initial_epoch=10)

Epoch 11/20
740/740 [==============================] - 461s - loss: 155819.0531 - val_loss: 319982.9104
Epoch 12/20
740/740 [==============================] - 458s - loss: 149556.8179 - val_loss: 326439.1328
Epoch 13/20
740/740 [==============================] - 457s - loss: 137595.8596 - val_loss: 308592.0021
Epoch 14/20
740/740 [==============================] - 457s - loss: 125835.9744 - val_loss: 307338.9342
Epoch 15/20
740/740 [==============================] - 456s - loss: 125583.4552 - val_loss: 338254.5018
Epoch 16/20
740/740 [==============================] - 458s - loss: 136610.4603 - val_loss: 304215.6111
Epoch 17/20
740/740 [==============================] - 456s - loss: 124356.0760 - val_loss: 309047.5350
Epoch 18/20
740/740 [==============================] - 461s - loss: 122048.7412 - val_loss: 323702.5492
Epoch 19/20
740/740 [==============================] - 456s - loss: 120002.4427 - val_loss: 320252.8305
Epoch 20/20
740/740 [==============================] - 459s - lo

In [20]:
model.fit_generator(train_batch, train_steps, epochs=30, verbose=1, callbacks=[csv_logger, checkpoint],
                    validation_data=val_batch, validation_steps=val_steps, initial_epoch=20)

Epoch 21/30
740/740 [==============================] - 638s - loss: 156921.5541 - val_loss: 310367.1075
Epoch 22/30
 60/740 [=>............................] - ETA: 534s - loss: 177604.9941

KeyboardInterrupt: 

In [15]:
model.fit_generator(train_batch, train_steps, epochs=5, verbose=1, callbacks=[reduce_lr],
                    validation_data=val_batch, validation_steps=val_steps)

Epoch 1/5
740/740 [==============================] - 500s - loss: 527431.7331 - val_loss: 521278.3349
Epoch 2/5
740/740 [==============================] - 572s - loss: 432504.8674 - val_loss: 457112.9539
Epoch 3/5
740/740 [==============================] - 493s - loss: 348094.1218 - val_loss: 362387.3584
Epoch 4/5
740/740 [==============================] - 492s - loss: 283841.3567 - val_loss: 315356.2590
Epoch 5/5
740/740 [==============================] - 495s - loss: 234211.2699 - val_loss: 299404.9821


In [16]:
model.fit_generator(train_batch, train_steps, epochs=5, verbose=1, callbacks=[reduce_lr],
                    validation_data=val_batch, validation_steps=val_steps)

Epoch 1/5
740/740 [==============================] - 498s - loss: 194996.2776 - val_loss: 281851.8154
Epoch 2/5
740/740 [==============================] - 495s - loss: 184835.2337 - val_loss: 276382.1604
Epoch 3/5
740/740 [==============================] - 496s - loss: 173231.3565 - val_loss: 294773.8319
Epoch 4/5
740/740 [==============================] - 495s - loss: 165042.1656 - val_loss: 296033.7463
Epoch 5/5
740/740 [==============================] - 641s - loss: 165090.1396 - val_loss: 318638.1986


In [22]:
# reduce lr
adam = Adam(lr = 5e-4)
model.compile(optimizer=adam, loss="mse") 

In [25]:
model.fit_generator(train_batch, train_steps, epochs=20, verbose=1, callbacks=[csv_logger, reduce_lr],
                    validation_data=val_batch, validation_steps=val_steps, initial_epoch=15)

Epoch 16/20
740/740 [==============================] - 684s - loss: 152989.1501 - val_loss: 304655.1528
Epoch 17/20
740/740 [==============================] - 521s - loss: 143858.3735 - val_loss: 288153.3071
Epoch 18/20
740/740 [==============================] - 455s - loss: 128485.5851 - val_loss: 287172.9297
Epoch 19/20
740/740 [==============================] - 457s - loss: 125270.3801 - val_loss: 300637.5496
Epoch 20/20
740/740 [==============================] - 456s - loss: 128174.6031 - val_loss: 303096.1404


In [28]:
# reduce lr
lr = K.get_value(model.optimizer.lr)
new_lr = 0.5 * lr
K.set_value(model.optimizer.lr, new_lr) 

In [30]:
model.fit_generator(train_batch, train_steps, epochs=25, verbose=1, callbacks=[csv_logger, reduce_lr],
                    validation_data=val_batch, validation_steps=val_steps, initial_epoch=20)

Epoch 21/25
 70/740 [=>............................] - ETA: 368s - loss: 292500.9789

KeyboardInterrupt: 

Save the keras model

In [30]:
model_file_name = cur_file + '_005' + '.h5'
model.save(os.path.join(model_path, model_file_name))

In [14]:
model_original.fit_generator(train_batch, train_steps, epochs=20, verbose=1, callbacks=[reduce_lr],
                             validation_data=val_batch, validation_steps=val_steps)

Epoch 1/20
740/740 [==============================] - 560s - loss: 360029.8089 - val_loss: 349874.1435
Epoch 2/20
740/740 [==============================] - 502s - loss: 218050.1641 - val_loss: 316249.5797
Epoch 3/20
740/740 [==============================] - 403s - loss: 184327.1608 - val_loss: 295999.1776
Epoch 4/20
740/740 [==============================] - 441s - loss: 179153.1940 - val_loss: 296833.6694
Epoch 5/20
740/740 [==============================] - 404s - loss: 147609.8334 - val_loss: 276048.3012
Epoch 6/20
740/740 [==============================] - 411s - loss: 144071.2798 - val_loss: 275369.3003
Epoch 7/20
345/740 [============>.................] - ETA: 259s - loss: 189607.1854

KeyboardInterrupt: 

In [18]:
adam = Adam(lr = 1e-5)
model_original.compile(optimizer=adam, loss="mse") 

In [19]:
model_original.fit_generator(train_batch, train_steps, epochs=1, verbose=1, callbacks=[reduce_lr],
                             validation_data=val_batch, validation_steps=val_steps)

Epoch 1/1
 55/740 [=>............................] - ETA: 319s - loss: 153433.9568

KeyboardInterrupt: 

In [16]:
X_train = np.array([img_to_arr(os.path.join(img_front_dir_path, p)) for p in df_train['front'][:2000]])
y_train = df_train['wheel-axis'][:2000].values

In [ ]:
model_original.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 6s - loss: 1467813.0381 - val_loss: 1148158.8000
Epoch 2/20
1600/1600 [==============================] - 6s - loss: 872211.1012 - val_loss: 976343.5500
Epoch 3/20
1600/1600 [==============================] - 6s - loss: 577758.3800 - val_loss: 574352.2562
Epoch 4/20
1600/1600 [==============================] - 6s - loss: 446692.7306 - val_loss: 415777.2825
Epoch 5/20
1600/1600 [==============================] - 6s - loss: 306212.0061 - val_loss: 425636.7406
Epoch 6/20
 192/1600 [==>...........................] - ETA: 5s - loss: 426554.9948

### Use batch normalization

Instead of normalizing the input ourselves, let's add a batch normalization layer as the first layer.

While we're at it, let's add bactch norm after each layer.

In [34]:
def get_model_batch_norm(input_shape):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=input_shape),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    
    adam = Adam(lr = 5e-4)
    model.compile(optimizer=adam, loss="mse") 
    model.summary()
    return model

In [35]:
model_batch_norm = get_model_batch_norm(input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 341, 562)       12        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 169, 279)      1824      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 169, 279)      96        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 36, 83, 138)       21636     
_________________________________________________________________
batch_normalization_3 (Batch (None, 36, 83, 138)       144       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 40, 67)        43248     
_________________________________________________________________
batch_normalization_4 (Batch (None, 48, 40, 67)        192       
__________

In [ ]:
model_batch_norm.fit_generator(train_batch, train_steps, epochs=20, verbose=1, callbacks=[reduce_lr],
                             validation_data=val_batch, validation_steps=val_steps)

Epoch 1/20
100/100 [==============================] - 23s - loss: 1595093.3869 - val_loss: 365298.0964
Epoch 2/20
100/100 [==============================] - 22s - loss: 1537063.8619 - val_loss: 370623.2904
Epoch 3/20
 45/100 [============>.................] - ETA: 8s - loss: 1575651.6201

In [12]:
model_batch_norm.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 21s - loss: 0.0624 - val_loss: 0.0286
Epoch 2/20
9742/9742 [==============================] - 21s - loss: 0.0183 - val_loss: 0.0167
Epoch 3/20
9742/9742 [==============================] - 21s - loss: 0.0146 - val_loss: 0.0162
Epoch 4/20
9742/9742 [==============================] - 21s - loss: 0.0127 - val_loss: 0.0157
Epoch 5/20
9742/9742 [==============================] - 21s - loss: 0.0102 - val_loss: 0.0155
Epoch 6/20
9742/9742 [==============================] - 21s - loss: 0.0081 - val_loss: 0.0165
Epoch 7/20
9742/9742 [==============================] - 21s - loss: 0.0068 - val_loss: 0.0159
Epoch 8/20
9742/9742 [==============================] - 21s - loss: 0.0058 - val_loss: 0.0147
Epoch 9/20
9742/9742 [==============================] - 21s - loss: 0.0051 - val_loss: 0.0154
Epoch 10/20
9742/9742 [==============================] - 21s - loss: 0.0048 - val_loss: 0.0150
Epoch 11/20

In [13]:
model_batch_norm.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 21s - loss: 0.0036 - val_loss: 0.0143
Epoch 2/20
9742/9742 [==============================] - 21s - loss: 0.0028 - val_loss: 0.0143
Epoch 3/20
9742/9742 [==============================] - 21s - loss: 0.0026 - val_loss: 0.0141
Epoch 4/20
9742/9742 [==============================] - 21s - loss: 0.0023 - val_loss: 0.0139
Epoch 5/20
9742/9742 [==============================] - 21s - loss: 0.0021 - val_loss: 0.0139
Epoch 6/20
9742/9742 [==============================] - 21s - loss: 0.0019 - val_loss: 0.0141
Epoch 7/20
9742/9742 [==============================] - 21s - loss: 0.0019 - val_loss: 0.0139
Epoch 8/20
9742/9742 [==============================] - 21s - loss: 0.0018 - val_loss: 0.0139
Epoch 9/20
9728/9742 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 00008: reducing learning rate to 1.0000000475e-05.
9742/9742 [==============================] - 21s - loss: 0.0017 

As we can see, batch norm didn't help.